<a href="https://colab.research.google.com/github/AlexKressner/WS25_Supply_Chain_Optimierung/blob/main/Fallstudie_Juicy_AG_Erweiterung_Beschaffung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fallstudie JUICY AG (Network Design), Erweiterung um Beschaffungsmärkte für Vorprodukte

In [1]:
! pip install -q pyscipopt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.2/17.2 MB 18.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
from pyscipopt import Model, quicksum

## Repo & Daten laden

In [3]:
! git clone https://github.com/AlexKressner/WS25_Supply_Chain_Optimierung

Cloning into 'WS25_Supply_Chain_Optimierung'...
remote: Enumerating objects: 155, done.
remote: Counting objects: 100% (155/155), done.
remote: Compressing objects: 100% (151/151), done.
Receiving objects: 100% (155/155), 183.78 KiB | 6.81 MiB/s, done.
remote: Total 155 (delta 74), reused 3 (delta 0), pack-reused 0 (from 0)
Resolving deltas: 100% (74/74), done.


In [4]:
! ls

sample_data  WS25_Supply_Chain_Optimierung


In [5]:
! ls WS25_Supply_Chain_Optimierung/Daten/NetworkDesign/

NetworkDesign_Beschaffungskosten.csv  NetworkDesign_Nachfrage.csv
NetworkDesign_Invest_Kapa.csv	      NetworkDesign_Produktionskosten.csv


In [6]:
folder = "WS25_Supply_Chain_Optimierung/Daten/NetworkDesign"

In [7]:
operative_kosten = pd.read_csv(f"{folder}/NetworkDesign_Produktionskosten.csv", sep=";")

In [8]:
operative_kosten.tail()

,Produktionsstandort,Markt,Produktionskosten
20,East London,USA,142
21,East London,Südamerika,100
22,East London,Europa,103
23,East London,Asien,105
24,East London,Afrika,71


In [9]:
operative_kosten.head()

,Produktionsstandort,Markt,Produktionskosten
0,Charleston,USA,81
1,Charleston,Südamerika,92
2,Charleston,Europa,101
3,Charleston,Asien,130
4,Charleston,Afrika,115


In [10]:
kapazitaeten = pd.read_csv(f"{folder}/NetworkDesign_Invest_Kapa.csv", sep=";")

In [11]:
kapazitaeten.head()

,Produktionsstandort,Ausbaustufe,Kapazitäten,Investitionen
0,Charleston,klein,10,6000
1,Charleston,groß,20,9000
2,Curitiba,klein,10,4500
3,Curitiba,groß,20,6750
4,Hamburg,klein,10,6500


In [12]:
nachfrage = pd.read_csv(f"{folder}/NetworkDesign_Nachfrage.csv", sep=";")

In [13]:
nachfrage

,Markt,Nachfragemenge
0,USA,12
1,Südamerika,8
2,Europa,14
3,Asien,16
4,Afrika,7


In [14]:
path = f"{folder}/NetworkDesign_Beschaffungskosten.csv"

In [15]:
beschaffung = pd.read_csv(path, sep=";")

In [16]:
beschaffung.head()

,Beschaffungsmarkt,Vorprodukt,Beschaffungskosten,Kapazitäten
0,Afrika,Flasche,10,60
1,Afrika,Vitaminwasser,20,30
2,Asien,Flasche,12,100
3,Asien,Vitaminwasser,15,40
4,Europa,Flasche,15,60


## Indexmengen

In [17]:
I = kapazitaeten.Produktionsstandort.unique()
J = list(nachfrage.Markt.unique())
A = kapazitaeten.Ausbaustufe.unique()

In [18]:
M = beschaffung["Beschaffungsmarkt"].unique().tolist() # Menge der Beschaffungsmärkte

In [19]:
VP = beschaffung["Vorprodukt"].unique().tolist() # Menge der Vorprodukte

## Parameter

In [20]:
# Produktions- und Transportkosten
operative_kosten.set_index(["Produktionsstandort","Markt"], inplace=True)
cp = operative_kosten.to_dict("dict")["Produktionskosten"]

In [21]:
# Fixkosten
cf = kapazitaeten.set_index(["Produktionsstandort","Ausbaustufe"])
cf = cf.to_dict("dict")["Investitionen"]

In [22]:
# Kapazitäten
cap = kapazitaeten.set_index("Ausbaustufe")
cap = cap.to_dict("dict")["Kapazitäten"]
cap

{'klein': 10, 'groß': 20}

In [23]:
# Nachfrage
nachfrage.set_index("Markt", inplace=True)
d = nachfrage.to_dict("dict")["Nachfragemenge"]
d


{'USA': 12, 'Südamerika': 8, 'Europa': 14, 'Asien': 16, 'Afrika': 7}

In [24]:
# Beschaffungskosten und Kapazitäten
beschaffung.set_index(["Beschaffungsmarkt","Vorprodukt"], inplace=True)

In [25]:
cb = beschaffung.to_dict("dict")["Beschaffungskosten"]
cb

{('Afrika', 'Flasche'): 10,
 ('Afrika', 'Vitaminwasser'): 20,
 ('Asien', 'Flasche'): 12,
 ('Asien', 'Vitaminwasser'): 15,
 ('Europa', 'Flasche'): 15,
 ('Europa', 'Vitaminwasser'): 20,
 ('Südamerika', 'Flasche'): 8,
 ('Südamerika', 'Vitaminwasser'): 10,
 ('USA', 'Flasche'): 11,
 ('USA', 'Vitaminwasser'): 6}

In [26]:
bcap = beschaffung.to_dict("dict")["Kapazitäten"]
bcap

{('Afrika', 'Flasche'): 60,
 ('Afrika', 'Vitaminwasser'): 30,
 ('Asien', 'Flasche'): 100,
 ('Asien', 'Vitaminwasser'): 40,
 ('Europa', 'Flasche'): 60,
 ('Europa', 'Vitaminwasser'): 60,
 ('Südamerika', 'Flasche'): 40,
 ('Südamerika', 'Vitaminwasser'): 30,
 ('USA', 'Flasche'): 50,
 ('USA', 'Vitaminwasser'): 35}

## Modellinstanz ziehen

In [27]:
model = Model()

## Entscheidungsvariablen

In [28]:
Y = {}
for i in I:
  for a in A:
    Y[i,a] = model.addVar(vtype="B", name=f"Ausbaustufe_{a}_Standort_{i}")

In [29]:
X = {}
for i in I:
  for j in J:
    X[i,j] = model.addVar(vtype="I", name=f"Produktion_Standort_{i}_für_Markt_{j}")

In [31]:
Z={}
for m in M:
  for p in VP:
    for i in I:
      Z[m,p,i] = model.addVar(vtype="C", name=f"{m},{p},{i}")

In [32]:
len(model.getVars())

85

## Zielfunktion

In [33]:
# Maximierung der gesamten Deckungsbeiträge
model.setObjective(
    quicksum(cp[i,j]*X[i,j] for i in I for j in J) + #Produktionskosten
    quicksum(cf[i,a]*Y[i,a] for i in I for a in A) + #Invest
    quicksum(cb[m,p]*Z[m,p,i] for m in M for p in VP for i in I), #Beschaffungskosten
    sense="minimize"
    )

## Nebenbedingungen

In [34]:
# Nachfrage (5)
for j in J:
  model.addCons(quicksum(X[i,j] for i in I) >= d[j], name=f"Nachfrage_Markt_{j}")

In [35]:
# Kopplung Produktion und Investment (5)
for i in I:
  model.addCons(
      quicksum(X[i,j] for j in J) <= quicksum(cap[a]*Y[i,a] for a in A),
      name=f"Kapazität_Standort_{i}"
      )

In [36]:
# Nur eine Ausbaustufe (5)
for i in I:
  model.addCons(
      quicksum(Y[i,a] for a in A) <= 1,
      name=f"Ausbaustufe_{i}"
      )

In [37]:
# Beschränkte Beschaffungskapazitäten
for m in M:
  for p in VP:
    model.addCons(quicksum(Z[m,p,i] for i in I)<= bcap[m,p])

In [38]:
# Flusserhaltung
for p in VP:
  for i in I:
    model.addCons(quicksum(Z[m,p,i] for m in M) == quicksum(X[i,j] for j in J))

In [39]:
len(model.getConss())

35

## Lösung

In [40]:
model.redirectOutput()

In [41]:
model.optimize()

presolving:
(round 1, fast)       0 del vars, 0 del conss, 0 add conss, 75 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 5 clqs, 0 implints
(round 2, exhaustive) 0 del vars, 0 del conss, 0 add conss, 75 chg bounds, 0 chg sides, 0 chg coeffs, 5 upgd conss, 0 impls, 5 clqs, 0 implints
   (0.0s) probing cycle finished: starting next cycle
(round 3, exhaustive) 0 del vars, 0 del conss, 0 add conss, 100 chg bounds, 0 chg sides, 0 chg coeffs, 5 upgd conss, 90 impls, 5 clqs, 0 implints
   (0.0s) probing cycle finished: starting next cycle
   (0.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.0s) no symmetry present (symcode time: 0.00)
presolving (4 rounds: 4 fast, 3 medium, 3 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 100 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 90 implications, 5 cliques, 0 implied integral variables (0 bin, 0 int, 0 cont)
presolved problem

In [42]:
print('LÖSUNG:')
print('Zielfunktionswert (Kosten) =', model.getObjVal())
print("\n")
print("PRODUKTIONS-MARKT-ZUTEILUNG")
for j in J:
  print(f"{j}:")
  for i in I:
    if model.getVal(X[i,j])>0:
      print('\t', f'{i,j} =', round(model.getVal(X[i,j])))
print("\n")
print("INVESTITIONEN")
for i in I:
  for a in A:
    if model.getVal(Y[i,a])>0:
      print(f'{i,a} =', round(model.getVal(Y[i,a])))
print("\n")
print("Beschaffungsmengen")
for i in I:
  for p in VP:
    for m in M:
      if model.getVal(Z[m,p,i]):
        print(f'{i,p,m} =', round(model.getVal(Z[m,p,i])))

LÖSUNG:
Zielfunktionswert (Kosten) = 24671.0


PRODUKTIONS-MARKT-ZUTEILUNG
USA:
	 ('Curitiba', 'USA') = 12
Südamerika:
	 ('Curitiba', 'Südamerika') = 8
Europa:
	 ('Quanzhou', 'Europa') = 4
	 ('East London', 'Europa') = 10
Asien:
	 ('Quanzhou', 'Asien') = 16
Afrika:
	 ('East London', 'Afrika') = 7


INVESTITIONEN
('Curitiba', 'groß') = 1
('Quanzhou', 'groß') = 1
('East London', 'groß') = 1


Beschaffungsmengen
('Curitiba', 'Flasche', 'Afrika') = 17
('Curitiba', 'Flasche', 'Südamerika') = 3
('Curitiba', 'Vitaminwasser', 'Südamerika') = 5
('Curitiba', 'Vitaminwasser', 'USA') = 15
('Quanzhou', 'Flasche', 'Südamerika') = 20
('Quanzhou', 'Vitaminwasser', 'USA') = 20
('East London', 'Flasche', 'Südamerika') = 17
('East London', 'Vitaminwasser', 'Südamerika') = 17


In [ ]:
model.getStatus()